In [38]:
from scipy.io import wavfile
from matplotlib import pyplot as plt
import wave
import pyaudio
import os

In [39]:
'''
Anger (ANG)
Disgust (DIS)
Fear (FEA)
Happy/Joy (HAP)
Neutral (NEU)
Sad (SAD)
'''
emotions = dict()
emotions["ANG"]=0
emotions["DIS"]=0
emotions["FEA"]=0
emotions["HAP"]=0
emotions["NEU"]=0
emotions["SAD"]=0

In [40]:

paths = f"{os.getcwd()}\\..\\Datasets\\AudioWav"
dir_list = os.listdir(paths)
for path in dir_list:
    emotions[path[9:12]]+=1

print(emotions)
# prints all files


{'ANG': 1271, 'DIS': 1271, 'FEA': 1271, 'HAP': 1271, 'NEU': 1087, 'SAD': 1271}


In [43]:
'''
Anger (a)
Disgust (d)
Fear (f)
Happy (h)
Neutral (n)
Saddness (sa)
surprise (su)
'''
emotions = dict()
emotions["a"]=0
emotions["d"]=0
emotions["f"]=0
emotions["h"]=0
emotions["sa"]=0
emotions["su"]=0
emotions["n"]=0
#Emotion has been described psychologically in discrete categories: anger, disgust, fear, happiness, sadness and surprise. A neutral category is also added to provide recordings of 7 emotion categories.

In [46]:
paths = f"{os.getcwd()}\\..\\Datasets\\ALL"
dir_list = os.listdir(paths)
second_character = ""
for path in dir_list:
    second_character = path[4] if (path[4] == "a" or path[4] == "u") else ""
    emotions[path[3]+second_character]+=1

print(emotions)

{'a': 60, 'd': 60, 'f': 60, 'h': 60, 'sa': 60, 'su': 60, 'n': 120}


In [ ]:
#https://www.geeksforgeeks.org/python-list-files-in-a-directory/
#

In [130]:
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display as ipd
import librosa
import librosa.display
import os
import soundfile as sf
import json
import random
from sklearn import preprocessing
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np

In [100]:
def get_files_from_path(directory):
    path_files = []
    dir_list = os.listdir(directory)
    for path in dir_list:
        path_files.append(directory+"\\"+path)
    return path_files

In [125]:
def extract_paths_for_emotions_keys(emotions_code, files_path):
    paths = []
    emotions_set = set(emotions_code)
    for code_file in files_path:
        #print(emotions_set, code_file[85:88])
        if (code_file[85:88] in emotions_set):
            paths.append(code_file)
    return paths

def extract_paths_for_emotions_keys_savee(emotions_code, files_path):
    paths = []
    emotions_set = set(emotions_code)
    for code_file in files_path:
        if (code_file[74] in emotions_set):
            paths.append(code_file)
    return paths

In [105]:
def features_extractor(file_name):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    mfccs_features = librosa.feature.mfcc(y=audio,sr=sample_rate,n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
    return mfccs_scaled_features

In [106]:
def save_elements_in_json(examples_saved, name):
    json_files = []
    json_file = {}
    index = 0
    for file in examples_saved:
        json_file = {"id": index, "features":[str(elem) for elem in file[0]] ,"code":file[1], "path":file[2]}
        json_files.append(json_file)
        index += 1
    json_object = json.dumps(json_files)
    with open(f"{name}.json", "w") as outfile:
        outfile.write(json_object)

In [11]:
files_path = get_files_from_path(f"{os.getcwd()}\\..\\Datasets\\AudioWav")
emotions_code = ["NEU", "FEA","ANG"]
datas_files = extract_paths_for_emotions_keys(emotions_code, files_path)

In [126]:
files_path_s = get_files_from_path(f"{os.getcwd()}\\..\\Datasets\\ALL")
emotions_code_s = ["a", "f","n"]
datas_files_s = extract_paths_for_emotions_keys_savee(emotions_code_s, files_path_s) 

In [168]:
examples = []
examples_saved = []
files_filters = dict()
files_filters["NEU"] = 0
files_filters["FEA"] = 0
files_filters["ANG"] = 0

for i in datas_files:
    file_stats = os.stat(i)
    #print(f'File Size in Bytes is {file_stats.st_size}', i[85:88])
    if (file_stats.st_size > 44):
        feature = features_extractor(i)
        examples.append(feature)
        files_filters[i[85:88]]+= 1
        examples_saved.append((feature, i[85:88], i))

In [169]:
print(len(examples))
print(files_filters)

3628
{'NEU': 1087, 'FEA': 1271, 'ANG': 1270}


In [141]:
examples_s = []
examples_saved_s = []
files_filters_s = dict()
files_filters_s["a"] = 0
files_filters_s["f"] = 0
files_filters_s["n"] = 0

for i in datas_files_s:
    file_stats_s = os.stat(i)
    #print(f'File Size in Bytes is {file_stats.st_size}', i[85:88])
    if (file_stats_s.st_size > 44):
        feature_s = features_extractor(i)
        examples_s.append(feature_s)
        files_filters_s[i[74]]+= 1
        examples_saved_s.append((feature_s, i[74], i))

In [142]:
print(len(examples_s))
print(files_filters_s)

240
{'a': 60, 'f': 60, 'n': 120}


In [135]:
def load_elements(name):
    f = open(f'{name}.json')
    data = json.load(f)
    examples = []
    for element in data:
        examples.append(([float(feature) for feature in (element["features"])], element["code"]))
    return examples

examples = load_elements("sample")

In [136]:
def select_elements(examples, code, quantity, new_code):
    random.shuffle(examples)
    elements = []
    counter = 1
    for example in examples:
        if (counter > quantity):
            break
        if code == example[1]:
            elements.append((example[0],new_code))
            counter = counter + 1
    return elements
        

In [145]:
examples_s = [] 
for example in examples_saved_s:
    examples_s.append((example[0], example[1]))

In [146]:
datas = select_elements(examples, 'NEU', 896,"without_stress")+select_elements(examples, 'ANG', 550, "stress")+select_elements(examples, 'FEA', 550, "stress")+select_elements(examples_s, 'a', 60, "stress")+select_elements(examples_s, 'f', 60, "stress")+select_elements(examples_s, 'n', 120, "without_stress")
random.shuffle(datas)

In [149]:
X = []
y = []
for data in datas:
    X.append(data[0])
    y.append(data[1])

In [150]:
labelencoder=preprocessing.LabelEncoder()
y = to_categorical(labelencoder.fit_transform(y))

In [151]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.2,random_state=0)

In [152]:
print(len(X_train),len(X_test),len(y_train),len(y_test))

1788 448 1788 448


In [119]:
#"C:\\Users\\bacs2\\Downloads\\Taller De Grado\\Previous\\Datasets\\AudioWAV\\1001_DFA_ANG_XX.wav"